In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import trange

# hyperparameters
batch_size = 48
block_size = 384
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 48
n_layer = 6
dropout = 0.2
#-----------

# torch.manual_seed(1337)

text = open('../data/combined_files_new.txt', 'r', encoding='utf-8').read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: list(stoi[c] for c in s)
decode = lambda l: ''.join(itos[i] for i in l)

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]


#data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)
        out = wei @ v
        return out
    
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
        
    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer Block: communication followed by computation """
    
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x
    
# super simple bigram model
class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get the probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
    
model = BigramLanguageModel()
m = model.to(device)
model = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

11.060322 M parameters


In [2]:
model.load_state_dict(torch.load('../data/lana.pt'))

RuntimeError: Error(s) in loading state_dict for BigramLanguageModel:
	Missing key(s) in state_dict: "blocks.0.sa.heads.32.tril", "blocks.0.sa.heads.32.key.weight", "blocks.0.sa.heads.32.query.weight", "blocks.0.sa.heads.32.value.weight", "blocks.0.sa.heads.33.tril", "blocks.0.sa.heads.33.key.weight", "blocks.0.sa.heads.33.query.weight", "blocks.0.sa.heads.33.value.weight", "blocks.0.sa.heads.34.tril", "blocks.0.sa.heads.34.key.weight", "blocks.0.sa.heads.34.query.weight", "blocks.0.sa.heads.34.value.weight", "blocks.0.sa.heads.35.tril", "blocks.0.sa.heads.35.key.weight", "blocks.0.sa.heads.35.query.weight", "blocks.0.sa.heads.35.value.weight", "blocks.0.sa.heads.36.tril", "blocks.0.sa.heads.36.key.weight", "blocks.0.sa.heads.36.query.weight", "blocks.0.sa.heads.36.value.weight", "blocks.0.sa.heads.37.tril", "blocks.0.sa.heads.37.key.weight", "blocks.0.sa.heads.37.query.weight", "blocks.0.sa.heads.37.value.weight", "blocks.0.sa.heads.38.tril", "blocks.0.sa.heads.38.key.weight", "blocks

In [19]:
model.load_state_dict(torch.load('../data/lana_overfit.pt'))

<All keys matched successfully>

In [26]:
max_iters = 5000
for iter in trange(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


  0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=50)[0].tolist()))

	Her between civil war ?	In some won despublics and


In [15]:
myData = "Human: Hi how are you?\n\nLana: I'm doing well, how are you?\n\nHuman: I'm great! What did you do this weekend?\n\nLana: Not much. you?\n\nHuman: We went to the movies. It was fun!\n\nLana: That's awesome! How old are you?\n\nHuman:"
myData = torch.tensor(encode(myData), dtype=torch.long, device=device).view(1, len(myData))
print(decode(m.generate(myData, max_new_tokens=500)[0].tolist()))

Human: Hi how are you?

Lana: I'm doing well, how are you?

Human: I'm great! What did you do this weekend?

Lana: Not much. you?

Human: We went to the movies. It was fun!

Lana: That's awesome! How old are you?

Human:Hu" word with?

Lana: I Wanow is the main tright these rye1: ) is the U.S.. foud of of a steme cell in early 1, when series , whichtmown onlow cell act from Brigania 's problems received along as health by ownking guat '' , not , Austries than observer term `` up spokemous '' but before skill 1 by a results .	0
who made the pigm herockers	Near Whatesríful , the ski , but is a boundary operate in super .	0
who made the pig in vigores	His Release diction a buttorarief a widemenweell appearatur his


In [14]:
torch.save(model.state_dict(), '../data/lana_overfit.pt')